In [ ]:
from astropy.io import fits
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import IterativeImputer

In [ ]:
# открыть файлы с скрасными смещениями

data = fits.open('http://gal-03.sai.msu.ru/~vtoptun/photometry/rcsed_v2_clean.fits')

data.info()

In [ ]:
data = data[1].data

In [ ]:
cols = data.columns.names #559

In [ ]:
# конвертация в датафрейм
RCSED_V2 = pd.DataFrame(np.array(data).byteswap().newbyteorder())
display(RCSED_V2.head())

In [ ]:
ID_cols = list(RCSED_V2.filter(regex='ID'))

In [ ]:
ID_cols.append('ind')
ID_cols.append('recno_uzc')

In [ ]:
ID_cols

In [ ]:
RCSED_V2 = RCSED_V2[RCSED_V2.columns.drop(ID_cols)]

In [ ]:
cat_cols = ['SpecFile_wigglez', 'targetname_6df', 'SeqNum_2df', 'planid_lamost']

In [ ]:
RCSED_V2 = RCSED_V2[RCSED_V2.columns.drop(cat_cols)]

In [ ]:
len(RCSED_V2.columns)

In [ ]:
RCSED_V2.head()

In [ ]:
len_data = len(RCSED_V2)

In [ ]:
sum(np.array(RCSED_V2.OBJNO_deep2) != -2147483648) / len_data

In [ ]:
#replace NaN

In [ ]:
def null_treating(cols, null):
    for col in cols:
        if sum(np.array(RCSED_V2[col]) == null) / len(RCSED_V2) > 0.9:
            RCSED_V2.drop([col],axis=1, inplace=True)
            cols.remove(col)
        else:
            RCSED_V2[col].replace(null, np.nan) 

In [ ]:
#null = -2147483648
cols_NaN1 = ['mjd_sdss', 'q_z_2df', 'specid_6df', 'obsid_lamost', 'lmjd_lamost', 'mjd_lamost',
             'OBJNO_deep2', 'OBJNO_deep3']

In [ ]:
#null = -32768
cols_NaN2 = ['plate_sdss', 'quality_6df', 'f_z_lega_c', 'Q_wigglez', 'NQ_gama']

In [ ]:
#null = 255
cols_NaN3 = ['spid_lamost', 'fiberid_lamost']

In [ ]:
null_treating(cols_NaN1, -2147483648)

In [ ]:
null_treating(cols_NaN2, -32768)

In [ ]:
null_treating(cols_NaN3, 255)

In [ ]:
len(RCSED_V2.columns)

In [ ]:
#  Anomaly detection

top_values = []
nan = []

for col in RCSED_V2.columns:
    vc = RCSED_V2[col].value_counts(ascending=False).head(1)
    nan = (RCSED_V2[col].isna()).sum()
    total = nan
    if vc[vc.index[0]] >1000:
        total = vc[vc.index[0]]+nan
    top_values.append([vc.name, vc.index[0], vc[vc.index[0]], nan, total])
    
anomaly = pd.DataFrame(data=top_values, columns = ['column_name', 'anomaly_value', 'value_counts', 'nan', 'total'])

In [ ]:
anomaly.sort_values(by='total', ascending=False)[1:20]

In [ ]:
#4105516*0.9 = 3694964

col_anomaly = anomaly[anomaly.total>3694964].column_name.to_list()

In [ ]:
len(col_anomaly)

In [ ]:
RCSED_V2.drop(col_anomaly, axis=1, inplace=True)

In [ ]:
sns.distplot(RCSED_V2.w3sigmag_7_wise)
plt.title('w3sigmag_7_wise')
plt.show()

In [ ]:
len(np.unique(RCSED_V2.w3sigmag_7_wise))-2202889

**To do**
*IterativeImputer*
-> models each feature with missing values as a function of other features, and uses that estimate for imputation